# Playground for Class Imbalance Slides

- Stephen W. Thomas
- Used for MMA 869, MMAI 869, and GMMA 869

In [ ]:
import datetime
print(datetime.datetime.now())

In [2]:
import pandas as pd
pd.show_versions(as_json=False)

import sklearn
sklearn.__version__


INSTALLED VERSIONS
------------------
commit           : None
python           : 3.6.10.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
machine          : AMD64
processor        : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.0.3
numpy            : 1.18.5
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.1.1
setuptools       : 49.1.0.post20200704
Cython           : None
pytest           : 5.4.3
hypothesis       : None
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : 4.5.0
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.16.1
pandas_datareader: None
bs4              : 4.9.0
bottleneck       : None
fastparquet      : None
gcsfs            : None
lxml.etree       : 4.5.0


'0.23.1'

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Dataset

The following dataset is a well-known imbalanced dataset from Kaggle:
    https://www.kaggle.com/mlg-ulb/creditcardfraud/home?select=creditcard.csv

In [4]:
df = pd.read_csv('../data/creditcard_sample.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142403 entries, 0 to 142402
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    142403 non-null  int64  
 1   V1      142403 non-null  float64
 2   V2      142403 non-null  float64
 3   V3      142403 non-null  float64
 4   V4      142403 non-null  float64
 5   V5      142403 non-null  float64
 6   V6      142403 non-null  float64
 7   V7      142403 non-null  float64
 8   V8      142403 non-null  float64
 9   V9      142403 non-null  float64
 10  V10     142403 non-null  float64
 11  V11     142403 non-null  float64
 12  V12     142403 non-null  float64
 13  V13     142403 non-null  float64
 14  V14     142403 non-null  float64
 15  V15     142403 non-null  float64
 16  V16     142403 non-null  float64
 17  V17     142403 non-null  float64
 18  V18     142403 non-null  float64
 19  V19     142403 non-null  float64
 20  V20     142403 non-null  float64
 21  V21     14

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Take a random sample, and then create X, y variables, and then split into training and testing.

In [5]:
from sklearn.model_selection import train_test_split

# This dataset is huge, so let's take a sample to speed things up
# df = df.sample(frac=0.5, replace=False, random_state=1, axis=0)
X = df.drop(['Class'], axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=44)

In [6]:
y_train.value_counts()
y_test.value_counts()

0    113707
1       215
Name: Class, dtype: int64

0    28427
1       54
Name: Class, dtype: int64

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

# Helper function
def quick_evaluate_with_dt(X_train, X_test, y_train, y_test, name, balance_weights=False):
    
    cw = None
    if balance_weights == True:
        cw = 'balanced'
        
    clf = DecisionTreeClassifier(random_state=0, class_weight=cw)
    clf = RandomForestClassifier(random_state=0, n_estimators=100, class_weight=cw)
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy       = accuracy_score(y_test, y_pred)
    f1             = f1_score(y_test, y_pred)
    recall         = recall_score(y_test, y_pred)
    precision      = precision_score(y_test, y_pred)
    roc_auc        = roc_auc_score(y_test, y_pred)
    
    df = pd.DataFrame({"Method"    : [name],
                       "Neg"       : [tn + fn],
                       "True Neg"  : [tn],
                       "False Neg" : [fn],
                       "Pos"       : [tp + fp],
                       "TP"        : [tp],
                       "FP"        : [fp],
                       "Accuracy"  : [accuracy],
                       "Recall"    : [recall],
                       "Precision" : [precision],
                       "F1"        : [f1],
                       "AUC"       : [roc_auc],
                      })
    
    print(df)
    return df

In [8]:
evals = list()

In [9]:
X_train.shape
y_train.shape
np.bincount(y_train)

evals.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'None'))

(113922, 30)

(113922,)

array([113707,    215], dtype=int64)

  Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy    Recall  \
0   None  28435     28425         10   46  44   2  0.999579  0.814815   

   Precision    F1       AUC  
0   0.956522  0.88  0.907372  


In [10]:
X_train.shape
y_train.shape
np.bincount(y_train)

evals.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'Class Weights', balance_weights=True))

(113922, 30)

(113922,)

array([113707,    215], dtype=int64)

          Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy    Recall  \
0  Class Weights  28436     28425         11   45  43   2  0.999544  0.796296   

   Precision        F1       AUC  
0   0.955556  0.868687  0.898113  


In [11]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over Random'))

C:\Users\st50\AppData\Local\Continuum\anaconda3\envs\small_sklearn\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\st50\AppData\Local\Continuum\anaconda3\envs\small_sklearn\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\st50\AppData\Local\Continuum\anaconda3\envs\small_sklearn\lib\site-packages\sklearn\u

(227414, 30)

(227414,)

array([113707, 113707], dtype=int64)

        Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy    Recall  \
0  Over Random  28434     28425          9   47  45   2  0.999614  0.833333   

   Precision        F1       AUC  
0   0.957447  0.891089  0.916631  


In [12]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over SMOTE'))

C:\Users\st50\AppData\Local\Continuum\anaconda3\envs\small_sklearn\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(227414, 30)

(227414,)

array([113707, 113707], dtype=int64)

       Method    Neg  True Neg  False Neg  Pos  TP  FP  Accuracy   Recall  \
0  Over SMOTE  28428     28421          7   53  47   6  0.999544  0.87037   

   Precision        F1      AUC  
0   0.886792  0.878505  0.93508  


In [13]:
from imblearn.over_sampling import ADASYN

X_resampled, y_resampled = ADASYN(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Over ADASYN'))

C:\Users\st50\AppData\Local\Continuum\anaconda3\envs\small_sklearn\lib\site-packages\sklearn\utils\deprecation.py:86: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


(227450, 30)

(227450,)

array([113707, 113743], dtype=int64)

KeyboardInterrupt: 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Under Sample'))

In [ ]:
# Recall the actual distrition of the truth labels of the testing set.
y_test.value_counts()

evals_all = pd.concat([m for m in evals], axis = 0).reset_index()

evals_all = evals_all.drop(columns = "index",axis =1)
evals_all.sort_values(by=['F1'], ascending=False)

# Another Dataset

The dataset is also imbalanced, although less so. It comes from:

https://www.kaggle.com/uciml/pima-indians-diabetes-database

In [ ]:
df_d = pd.read_csv('data/diabetes_orig.csv')
df_d.info()
df_d.head()

X = df_d.drop(['Id', 'diabetes'], axis=1)
y = df_d['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=44)

In [ ]:
y_train.value_counts()
y_test.value_counts()

In [ ]:
evals_d = list()

In [ ]:
X_train.shape
y_train.shape
np.bincount(y_train)

evals_d.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'None'))

In [ ]:
X_train.shape
y_train.shape
np.bincount(y_train)

evals_d.append(quick_evaluate_with_dt(X_train, X_test, y_train, y_test, 'Class Weights', balance_weights=True))

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals_d.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over Random'))

In [ ]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals_d.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test, 'Over SMOTE'))

In [ ]:
from imblearn.over_sampling import ADASYN

X_resampled, y_resampled = ADASYN(random_state=0).fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals_d.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Over ADASYN'))

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

X_resampled.shape
y_resampled.shape
np.bincount(y_resampled)

evals_d.append(quick_evaluate_with_dt(X_resampled, X_test, y_resampled, y_test,  'Under Sample'))

In [ ]:
# Recall the actual distrition of the truth labels of the testing set.
y_test.value_counts()

evals_d_all = pd.concat([m for m in evals_d], axis = 0).reset_index()

evals_d_all = evals_d_all.drop(columns = "index",axis =1)
evals_d_all.sort_values(by=['F1'], ascending=False)